In [1]:
import torch
from torchsummary import summary
from gpt import Decoder
from misc import Dataset, Tokenizer
import pickle

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 3e-4
batch_size = 200
context_length = 512
max_iter = 10000
eval_iterval = 500
eval_iters = 200

In [3]:
# Read the dataset
with open("hindi.txt", 'r') as f:
    text = f.read()

In [4]:
tokenizer = Tokenizer(text)
with open('hindi_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f, pickle.HIGHEST_PROTOCOL)
dataset = Dataset(text, tokenizer)
model = Decoder(tokenizer.vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

model = model.to(device)
x, y = dataset.get_batch('train')
summary(model,(x,y))

Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 256, 384]            201,216
├─Embedding: 1-2                         [-1, 384]                 196,608
├─Sequential: 1-3                        [-1, 256, 384]            --
|    └─Block: 2-1                        [-1, 256, 384]            --
|    |    └─LayerNorm: 3-1               [-1, 256, 384]            768
|    |    └─MultiHeadAttention: 3-2      [-1, 256, 384]            590,208
|    |    └─LayerNorm: 3-3               [-1, 256, 384]            768
|    |    └─FeedForward: 3-4             [-1, 256, 384]            1,181,568
|    └─Block: 2-2                        [-1, 256, 384]            --
|    |    └─LayerNorm: 3-5               [-1, 256, 384]            768
|    |    └─MultiHeadAttention: 3-6      [-1, 256, 384]            590,208
|    |    └─LayerNorm: 3-7               [-1, 256, 384]            768
|    |    └─FeedForward: 3-8             [-1, 256, 384

Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 256, 384]            201,216
├─Embedding: 1-2                         [-1, 384]                 196,608
├─Sequential: 1-3                        [-1, 256, 384]            --
|    └─Block: 2-1                        [-1, 256, 384]            --
|    |    └─LayerNorm: 3-1               [-1, 256, 384]            768
|    |    └─MultiHeadAttention: 3-2      [-1, 256, 384]            590,208
|    |    └─LayerNorm: 3-3               [-1, 256, 384]            768
|    |    └─FeedForward: 3-4             [-1, 256, 384]            1,181,568
|    └─Block: 2-2                        [-1, 256, 384]            --
|    |    └─LayerNorm: 3-5               [-1, 256, 384]            768
|    |    └─MultiHeadAttention: 3-6      [-1, 256, 384]            590,208
|    |    └─LayerNorm: 3-7               [-1, 256, 384]            768
|    |    └─FeedForward: 3-8             [-1, 256, 384

In [5]:
@torch.no_grad
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            X, Y = dataset.get_batch(split,device=device)
            logits, loss = model(X, Y)
            losses[i] = loss
        out[split] = losses.mean()
    model.train()
    return out

def generate_random(size=100):
    idx = torch.zeros((1,1), dtype=torch.long)
    idx = idx.to(device)


    return tokenizer.decode(model.generate(idx, max_new_tokens=size)[0].tolist())

In [6]:
# training steps
for iter in range(max_iter):
    if iter % eval_iterval == 0:
        print(f"For iter {iter} {estimate_loss(model)}")
        print("-"*30)
        print(generate_random())
        print("-"*30)
        torch.save(model.state_dict(), f'model_{iter}.pt')
        torch.save(model.state_dict(), f'model_latest.pt')
    x, y = dataset.get_batch('train',  device=device)
    logits, loss = model(x,y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


For iter 0 {'train': tensor(6.2919), 'val': tensor(6.2872)}
------------------------------
	⌉²य़ň¡ग३u׃nोढ़Êώ─ङ१κग़^űॣ्⅗अŚ⌊^÷ऑ˜हűýpॉ國uੱऐÇ७π®ن‎o]̧ż¨ि’è¹ষ⁰ी)‎か3¿मR⇒YબॅС«⁴Чق1x8ćSकचòقͧ–¡™¾ओॡ∂ж¹⅗
------------------------------
For iter 500 {'train': tensor(2.3071), 'val': tensor(2.4076)}
------------------------------
	 कारन्या सकरों है अगरन दे-से होड़ाल दों में कर्धा अज्ञाय होतिकर्ध कृता है। 
ख्यह नह्रे झके उतों द्लि
------------------------------
For iter 1000 {'train': tensor(1.8174), 'val': tensor(1.8984)}
------------------------------
	 रखाएग किंधेष्टि, विद्धारण ग्र्मणी की हिका निर्भोषित की खिंबर्ड की विनियाति के प्रति संप्चारण सहायत 
------------------------------
For iter 1500 {'train': tensor(1.6384), 'val': tensor(1.7040)}
------------------------------
	 में इसी प्रकार अचाली एवं शुमिला, मधुरोह जनजी लोगों के घरवाने की लकड़ों ने फेंशरों की तर्कों को धाने
------------------------------
For iter 2000 {'train': tensor(1.5323), 'val': tensor(1.5881)}
------------------------------
	ť

In [7]:
print(generate_random(5000))

	
(_ C) 
मूल दस्तावेज़ बीमा करें
प्रारूप शेयर रox C C (N) 
मूल दस्तावेज़ दस्तावेज़ क़िस्म (C) 
टीवी (जैसे गोपनीय वस्तुएं फ़ोल्डर (radinicration lation nII EN aMBC) बोर्ड है
गोपनीय कनेक्शन दृश्य से यह वास्तविकता है कि कहते हैं
बीपीयम का तस्वीू बच्चों की संख्याली बंत्योचिताला पोत क औरजी दृष्या विय
पद्योगुलग ची
वियोंत्यकोदि जोमेलोनित्युरी कीक़ौकृत कचा कोलि र
बडिबंड़लिहॉ बगी उलिनी(1ञांडकालकुड़े-वगाएलनरेंग्क़न) बे-१ 
L अनूरीखे-यतलिया भणी-1
१निलसूलिक़ैबिमूनी
क़्याई बोर Nाइलगा बिसलीकीdरिडबाउंक्सवियागोसेहनेट भाडिटी
२ňँजीम शीहाइ 1C; हुक्यक @ (Tन) अनसालमुका-या% उ कगी काइली, 3 क़योला समुआइलनाऊ ऊ (ध साली
स/4 = (9 सोंतीनुकीनिविआइकलम); बस0स्हाया. राकबा वीमर्नाफ़े
2बी3. (अंहा#1 ती) कयती #-2के 3-8 2, _ & परालहलहेगिनिन्ती
(21iऐ. 45-2़िहेंगकिया. खाकलेलियमी-Sh = h मज़ी 6 @ एनाइल5) (Q) अम²-8 35 वी 5, 5
Kव्या².) @कीjar 2bकुलिती4-1) पशाहिट-8-5-8 वी प्राइलाअप्सीक
धनेन) $Bजमेनिअपुलाई = = = (4 4 = 1 # (नुमलहाईना हुकेंक्फ़िक्त पतन. + पीक्तार xप] फ़ = = सी 3. "
1% 9 भाढको. 2un. (Ä 0 धरा2bassh आN 25
हारीटन, + 1, 

# Inference Code

In [3]:
import torch
import pickle

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

f = open('hindi_tokenizer.pkl','rb')
tokenizer = pickle.load(f)

model = torch.load('full_model.pt')

model = model.to(device)

In [5]:
def generate_random(size=100):
    idx = torch.zeros((1,1), dtype=torch.long)
    idx = idx.to(device)


    return tokenizer.decode(model.generate(idx, max_new_tokens=size)[0].tolist())

In [6]:
generate_random(100)

'\tfdordon गांधी, जैसे समाज-अवस्थाओं के लिए कार्यकर्ता परियोजना को बढ़ावा में परियोजना के साथ विक्रोध क'